## Motion correction

Motion correction has been performed using MotionCor2.

Please cite:

1. Zheng, S., Palovcak, E., Armache, JP. et al. MotionCor2: anisotropic correction of beam-induced motion for improved cryo-electron microscopy. *Nat Methods* **14**, 331–332 (2017).

In [ ]:
def read_patch_frame_into_df(patch_frame_fname):
    """Read a single patch log file"""
    # Read in log file
    patch_frame_df = pd.read_table(
        patch_frame_fname,
        names=["Frame", "patch_x", "patch_y", "x_shift", "y_shift"],
        comment="#",
        sep=" ",
        skipinitialspace=True,
    )

    # Assign unique index to each patch
    patch_idx = []
    for frame in set(patch_frame_df.Frame):
        for i in range(len(patch_frame_df[patch_frame_df.Frame == frame])):
            patch_idx.append(i)

    patch_frame_df["patch_idx"] = patch_idx

    return patch_frame_df


def get_ts_log_df(tilts_dir):
    """Get a df of the shifts from the patch log files"""
    tilts_list = []
    for f in os.listdir(tilts_dir):
        if f.endswith("Patch-Frame.log"):
            ta = f.split("_")[-1].split(".mrc")[0]
            ts = f.split("_")[1]
            df = read_patch_frame_into_df(f"{tilts_dir}/{f}")
            df = df.assign(TiltSeries=int(ts))
            df = df.assign(TiltAngle=float(ta))
            df = df.assign(EuclideanShift=(df.x_shift**2 + df.y_shift**2) ** 0.5)
            tilts_list.append(df)
        else:
            pass

    ts_df = pd.concat(tilts_list)
    return ts_df


def get_tilts_dir():
    """Get the directory where the patch log files are kept"""
    try:
        mc2_yaml = f"{os.getcwd()}/{proj_name}_mc2.yaml"
    except FileNotFoundError:
        print(f"{proj_name}_mc2.yaml not found.")

    with open(mc2_yaml, "r") as f:
        output = yaml.load(f.read(), Loader=yaml.FullLoader)
        tilts_dir = output["System"]["output_path"]

    return tilts_dir

### Summary of shifts for each tilt series

In [ ]:
ts_df = get_ts_log_df(get_tilts_dir())
# pd.set_option("display.max_rows", None)
display(
    ts_df.groupby("TiltSeries")["EuclideanShift"].describe()[
        ["mean", "std", "25%", "50%", "75%"]
    ]
)

In [ ]:
plt.figure(figsize=(15, 5))
sns.barplot(
    x="TiltSeries",
    y="EuclideanShift",
    data=ts_df,
    errorbar="sd",
    color="cornflowerblue",
)
plt.gca().set_ylim(bottom=0)
plt.ylabel("Euclidean Shift (px)")
plt.tight_layout()
plt.title("Euclidean Shift Means +/- Standard Deviation")